###### ODONGKARA OSCAR S23B23/085 B24774
###### NANKYA ZAHARAH 
###### OBBA MARK CALVIN

In [ ]:
import datetime
from typing import List, Dict
import tkinter as tk
from tkinter import ttk, messagebox
import json
from enum import Enum
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


class TaskType(Enum):
    ACADEMIC = "Academic"
    PERSONAL = "Personal"
    WORK = "Work"


class TaskPriority(Enum):
    HIGH = "High"
    MEDIUM = "Medium"
    LOW = "Low"


class Task:
    def __init__(self, title: str, task_type: TaskType, priority: TaskPriority,
                 start_time: datetime.datetime, end_time: datetime.datetime):
        self.title = title
        self.task_type = task_type
        self.priority = priority
        self.start_time = start_time
        self.end_time = end_time
        self.completed = False
        self.reminded = False

    def to_dict(self):
        return {
            "title": self.title,
            "task_type": self.task_type.value,
            "priority": self.priority.value,
            "start_time": self.start_time.isoformat(),
            "end_time": self.end_time.isoformat(),
            "completed": self.completed,
            "reminded": self.reminded
        }


class SchedulingAssistant:
    def __init__(self):
        self.tasks: List[Task] = []
        self.load_tasks()

    def add_task(self, task: Task) -> None:
        self.tasks.append(task)
        self.tasks = self.merge_sort(self.tasks)
        self.save_tasks()

    def save_tasks(self):
        with open("tasks.json", "w") as f:
            json.dump([task.to_dict() for task in self.tasks], f, indent=4)

    def load_tasks(self):
        try:
            with open("tasks.json", "r") as f:
                tasks = json.load(f)
                self.tasks = []
                for task_data in tasks:
                    task = Task(
                        title=task_data["title"],
                        task_type=TaskType(task_data["task_type"]),
                        priority=TaskPriority(task_data["priority"]),
                        start_time=datetime.datetime.fromisoformat(task_data["start_time"]),
                        end_time=datetime.datetime.fromisoformat(task_data["end_time"]),
                    )
                    task.completed = task_data.get("completed", False)
                    task.reminded = task_data.get("reminded", False)
                    self.tasks.append(task)
        except (FileNotFoundError, json.JSONDecodeError):
            self.tasks = []

    def maximize_tasks(self):
        priority_weights = {TaskPriority.HIGH: 3, TaskPriority.MEDIUM: 2, TaskPriority.LOW: 1}
        tasks = sorted(self.tasks, key=lambda x: x.end_time)

        n = len(tasks)
        dp = [0] * (n + 1)
        p = [0] * n

        def binary_search(j):
            low, high = 0, j - 1
            while low <= high:
                mid = (low + high) // 2
                if tasks[mid].end_time <= tasks[j].start_time:
                    if tasks[mid + 1].end_time <= tasks[j].start_time:
                        low = mid + 1
                    else:
                        return mid
                else:
                    high = mid - 1
            return -1

        for j in range(n):
            p[j] = binary_search(j)

        for j in range(1, n + 1):
            weight = priority_weights[tasks[j - 1].priority]
            dp[j] = max(dp[j - 1], weight + dp[p[j - 1] + 1])

        selected_tasks = []
        j = n
        while j > 0:
            weight = priority_weights[tasks[j - 1].priority]
            if weight + dp[p[j - 1] + 1] > dp[j - 1]:
                selected_tasks.append(tasks[j - 1])
                j = p[j - 1] + 1
            else:
                j -= 1

        selected_tasks.reverse()
        return selected_tasks

    def display_maximized_tasks(self):
        optimal_tasks = self.maximize_tasks()
        if not optimal_tasks:
            messagebox.showinfo("Maximized Tasks", "No tasks could be selected without overlap.")
            return

        result = "\n".join([
            f"{task.title} ({task.priority.value}) - {task.start_time.strftime('%Y-%m-%d %H:%M')} to {task.end_time.strftime('%Y-%m-%d %H:%M')}"
            for task in optimal_tasks
        ])
        messagebox.showinfo("Maximized Tasks", f"Optimal Tasks:\n\n{result}")

    def delete_task(self, title: str):
        self.tasks = [task for task in self.tasks if task.title != title]
        self.save_tasks()

    def send_reminders(self):
        now = datetime.datetime.now()
        for task in self.tasks:
            if not task.reminded and task.start_time - now <= datetime.timedelta(hours=24):
                messagebox.showinfo("Reminder", f"{task.title} is due soon!")
                task.reminded = True
        self.save_tasks()

    def get_busy_slots(self) -> List[Dict]:
        busy_slots = []
        for task in self.tasks:
            busy_slots.append({
                "start": task.start_time,
                "end": task.end_time,
                "load": (task.end_time - task.start_time).total_seconds() / 3600
            })
        return busy_slots

    def generate_gantt_chart(self):
        if not self.tasks:
            messagebox.showinfo("Gantt Chart", "No tasks to display")
            return

        fig, ax = plt.subplots(figsize=(15, 8))
        color_map = {
            TaskPriority.HIGH: 'red',
            TaskPriority.MEDIUM: 'orange',
            TaskPriority.LOW: 'green'
        }

        for i, task in enumerate(sorted(self.tasks, key=lambda x: x.start_time)):
            duration = (task.end_time - task.start_time).total_seconds() / 3600
            color = color_map.get(task.priority, 'blue')

            ax.barh(
                task.title,
                duration,
                left=mdates.date2num(task.start_time),
                height=0.5,
                color=color,
                alpha=0.7,
                edgecolor='black',
                linewidth=1
            )

        ax.xaxis_date()
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
        plt.gcf().autofmt_xdate()

        ax.set_title('Task Schedule Gantt Chart')
        ax.set_xlabel('Time')
        ax.set_ylabel('Tasks')
        ax.grid(True, axis='x', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

    def merge_sort(self, tasks: List[Task]) -> List[Task]:
        if len(tasks) <= 1:
            return tasks
        mid = len(tasks) // 2
        left_half = self.merge_sort(tasks[:mid])
        right_half = self.merge_sort(tasks[mid:])
        return self.merge(left_half, right_half)

    def merge(self, left: List[Task], right: List[Task]) -> List[Task]:
        sorted_tasks = []
        while left and right:
            if self.compare_tasks(left[0], right[0]) <= 0:
                sorted_tasks.append(left.pop(0))
            else:
                sorted_tasks.append(right.pop(0))
        sorted_tasks.extend(left)
        sorted_tasks.extend(right)
        return sorted_tasks

    def compare_tasks(self, task1: Task, task2: Task) -> int:
        priority_order = {TaskPriority.HIGH: 3, TaskPriority.MEDIUM: 2, TaskPriority.LOW: 1}
        if task1.priority != task2.priority:
            return priority_order[task1.priority] - priority_order[task2.priority]
        return (task1.start_time - task2.start_time).total_seconds()

class SchedulerGUI:
    def __init__(self, root):
        self.scheduler = SchedulingAssistant()
        self.root = root
        self.root.title("Personal Scheduling Assistant")
        self.root.geometry("1200x800")
        self.root.configure(bg="#F0F0F0")

        self.create_widgets()

    def create_widgets(self):
        # Main Frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Task List Frame
        task_list_frame = ttk.LabelFrame(main_frame, text="Task List")
        task_list_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # Treeview for Tasks
        self.tree = ttk.Treeview(task_list_frame, columns=(
            "Title", "Type", "Priority", "Start Time", "End Time", "Status"
        ), show='headings')
        
        for col in ["Title", "Type", "Priority", "Start Time", "End Time", "Status"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, anchor='center')
        
        self.tree.pack(fill=tk.BOTH, expand=True)

        # Delete Button
        delete_btn = ttk.Button(task_list_frame, text="Delete Selected Task", command=self.delete_selected_task)
        delete_btn.pack(pady=5)

        # Task Entry Frame
        entry_frame = ttk.LabelFrame(main_frame, text="Add New Task")
        entry_frame.pack(fill=tk.X, padx=5, pady=5)

        # Entry Fields
        fields = [
            ("Title:", "title_entry"),
            ("Type:", "type_combo"),
            ("Priority:", "priority_combo"),
            ("Start Time (YYYY-MM-DD HH:MM):", "start_time_entry"),
            ("End Time (YYYY-MM-DD HH:MM):", "end_time_entry")
        ]

        for label, attr in fields:
            frame = ttk.Frame(entry_frame)
            frame.pack(fill=tk.X, padx=5, pady=2)
            
            ttk.Label(frame, text=label).pack(side=tk.LEFT, padx=5)
            
            if attr == "type_combo":
                setattr(self, attr, ttk.Combobox(frame, values=[t.value for t in TaskType], state="readonly"))
            elif attr == "priority_combo":
                setattr(self, attr, ttk.Combobox(frame, values=[p.value for p in TaskPriority], state="readonly"))
            else:
                setattr(self, attr, ttk.Entry(frame, width=40))
            
            getattr(self, attr).pack(side=tk.LEFT, expand=True, fill=tk.X, padx=5)

        # Add Task Button
        add_task_btn = ttk.Button(entry_frame, text="Add Task", command=self.add_task)
        add_task_btn.pack(pady=5)

        # Analysis Frame
        analysis_frame = ttk.LabelFrame(main_frame, text="Task Analysis")
        analysis_frame.pack(fill=tk.X, padx=5, pady=5)

        analysis_buttons = [
            ("Generate Gantt Chart", self.scheduler.generate_gantt_chart),
            ("Send Reminders", self.scheduler.send_reminders),
            ("Analyze Busy Slots", self.analyze_busy_slots),
            ("Maximize Tasks", self.scheduler.display_maximized_tasks)
        ]

        for label, command in analysis_buttons:
            ttk.Button(analysis_frame, text=label, command=command).pack(side=tk.LEFT, padx=5, pady=5)

        # Initial update
        self.update_task_list()

    def add_task(self):
        try:
            task = Task(
                title=self.title_entry.get(),
                task_type=TaskType(self.type_combo.get()),
                priority=TaskPriority(self.priority_combo.get()),
                start_time=datetime.datetime.strptime(self.start_time_entry.get(), "%Y-%m-%d %H:%M"),
                end_time=datetime.datetime.strptime(self.end_time_entry.get(), "%Y-%m-%d %H:%M")
            )
            
            if task.start_time >= task.end_time:
                raise ValueError("End time must be after start time")
            
            self.scheduler.add_task(task)
            self.update_task_list()
            messagebox.showinfo("Success", "Task added successfully!")
        except ValueError as e:
            messagebox.showerror("Error", str(e))

    def delete_selected_task(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "No task selected")
            return
        
        task_title = self.tree.item(selected[0])['values'][0]
        self.scheduler.delete_task(task_title)
        self.update_task_list()

    def update_task_list(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        
        for task in self.scheduler.tasks:
            self.tree.insert("", "end", values=(
                task.title, 
                task.task_type.value, 
                task.priority.value,
                task.start_time.strftime("%Y-%m-%d %H:%M"),
                task.end_time.strftime("%Y-%m-%d %H:%M"),
                "Completed" if task.completed else "Pending"
            ))

    def analyze_busy_slots(self):
        busy_slots = self.scheduler.get_busy_slots()
        
        plt.figure(figsize=(10, 5))
        time_slots = [slot['start'].strftime("%Y-%m-%d %H:%M") for slot in busy_slots]
        loads = [slot['load'] for slot in busy_slots]
        
        plt.bar(time_slots, loads, color='skyblue')
        plt.title('Busy Time Slots')
        plt.xlabel('Time Slot')
        plt.ylabel('Load (Hours)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def main():
    root = tk.Tk()
    app = SchedulerGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()